# MapReduce for word count with streaming

I will examine the options for streaming that control the number of reducers. In the Cloudera Quickstart VM terminal, I will execute a simple word count example in Python 2.6. 

## Mapper for word count

wordcount_mapper.py

In [ ]:
#!/usr/bin/env python   
#the above just indicates to use python to intepret this file

# ---------------------------------------------------------------
#This mapper code will input a line of text and output <word, 1>
# 
# ---------------------------------------------------------------

import sys             #a python module with system functions for this OS

# ------------------------------------------------------------
#  this 'for loop' will set 'line' to an input line from system 
#    standard input file
# ------------------------------------------------------------
for line in sys.stdin:  

#-----------------------------------
#sys.stdin call 'sys' to read a line from standard input, 
# note that 'line' is a string object, ie variable, and it has methods that you can apply to it,
# as in the next line
# ---------------------------------
    line = line.strip()  #strip is a method, ie function, associated
                         #  with string variable, it will strip 
                         #   the carriage return (by default)
    keys = line.split()  #split line at blanks (by default), 
                         #   and return a list of keys
    for key in keys:     #a for loop through the list of keys
        value = 1        
        print('{0}\t{1}'.format(key, value) ) #the {} is replaced by 0th,1st items in format list
                            #also, note that the Hadoop default is 'tab' separates key from the value

## Reducer for word count

wordcount_reducer.py

In [ ]:
#!/usr/bin/env python

# ---------------------------------------------------------------
#This reducer code will input a line of text and 
#    output <word, total-count>
# ---------------------------------------------------------------
import sys

last_key      = None              #initialize these variables
running_total = 0

# -----------------------------------
# Loop thru file
#  --------------------------------
for input_line in sys.stdin:
    input_line = input_line.strip()

    # --------------------------------
    # Get Next Word    # --------------------------------
    this_key, value = input_line.split("\t", 1)  #the Hadoop default is tab separates key value
                          #the split command returns a list of strings, in this case into 2 variables
    value = int(value)           #int() will convert a string to integer (this program does no error checking)
 
    # ---------------------------------
    # Key Check part
    #    if this current key is same 
    #          as the last one Consolidate
    #    otherwise  Emit
    # ---------------------------------
    if last_key == this_key:       
        running_total += value   # add value to running total

    else:
        if last_key:             #if this key that was just read in
                                 #   is different, and the previous 
                                 #   (ie last) key is not empy,
                                 #   then output 
                                 #   the previous <key running-count>
            print( "{0}\t{1}".format(last_key, running_total) )
                                 # hadoop expects tab(ie '\t') 
                                 #    separation
        running_total = value    #reset values
        last_key = this_key

if last_key == this_key:
    print( "{0}\t{1}".format(last_key, running_total))

## Procedure

Copy and paste the above into a text file as follows from the terminal prompt in Cloudera VM.

Type in the following to open a text editor, and then cut and paste the above lines for wordcount_mapper.py into the text editor, save, and exit. Repeat for wordcount_reducer.py

In [ ]:
gedit wordcount_mapper.py
gedit wordcount_reducer.py

Enter the following to check that the indentations line up as above.

In [ ]:
more wordcount_mapper.py
more wordcount_reducer.py

Enter the following to make it executable. `chmod` means change mode. `chmod +x` <=> `chmod a+x` means make file executable for all users. `+x` means adding executable permission.

In [ ]:
chmod +x wordcount_mapper.py
chmod +x wordcount_reducer.py

Create some data:

In [ ]:
echo "A long time ago in a galaxy far far away" > /home/cloudera/testfile1
echo "Another episode of Star Wars" > /home/cloudera/testfile2

`echo` is to write its arguments in the bash to standard output. Syntax: `echo [option(s)] [string(s)] > filename`.

Create a directory on the HDFS file system (if already exists that’s OK):

In [ ]:
hdfs dfs -mkdir /user/cloudera/input

Copy the files from local filesystem to the HDFS filesystem:

In [ ]:
hdfs dfs -put /home/cloudera/testfile1 /user/cloudera/input

hdfs dfs -put /home/cloudera/testfile2 /user/cloudera/input

You can see your files on HDFS

In [ ]:
hdfs dfs -ls /user/cloudera/input

Run the Hadoop WordCount example with the input and output specified.

Note that your file paths may differ. The ‘\’ just means the command continues on next line.

In [ ]:
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/cloudera/input \
-output /user/cloudera/output_new \
-mapper /home/cloudera/wordcount_mapper.py \
-reducer /home/cloudera/wordcount_reducer.py

Hadoop prints out a whole lot of logging or error information. If it runs you will see something like the following on the screen scroll by:

In [ ]:
```

INFO mapreduce.Job: map 0% reduce 0%

INFO mapreduce.Job: map 67% reduce 0%

INFO mapreduce.Job: map 100% reduce 0%

INFO mapreduce.Job: map 100% reduce 100%

INFO mapreduce.Job: Job job_1442937183788_0003 completed successfully

```

Check the output file to see the results:

In [ ]:
hdfs dfs -cat /user/cloudera/output_new/part-00000

Note that since we have one reducer by default, we can see all the results in one file `part-00000`. However, if we have multiple reducers, the results will be in the multiple files. Then we can check files like `part-00001`.

View the output directory:

In [ ]:
hdfs dfs -ls /user/cloudera/output_new

Look at the files there and check out the contents, e.g.:

In [ ]:
hdfs dfs -cat /user/cloudera/output_new/part-00000

Remark: `cat` stands for "catenate". It reads data from files and outputs their contents. It is the simplest way to dispay the contents of a file at the command line.

Streaming options:

In [ ]:
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar --help

Let’s change the number of reduce tasks to see its effects. Setting it to 0 will execute no reducer and only produce the map output. (Note the output directory is changed in the snippet below because Hadoop doesn’t like to overwrite output)

In [ ]:
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/cloudera/input \
-output /user/cloudera/output_new_0 \
-mapper /home/cloudera/wordcount_mapper.py \
-reducer /home/cloudera/wordcount_reducer.py \
-numReduceTasks 0

Get the output file from this run, and then upload it to local:

In [ ]:
hdfs dfs -getmerge /user/cloudera/output_new_0/* wordcount_num0_output.txt

Change the number of reducers to 2 and answer the related quiz question at the end of the video lesson

In [ ]:
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/cloudera/input \
-output /user/cloudera/output_new_2 \
-mapper /home/cloudera/wordcount_mapper.py \
-reducer /home/cloudera/wordcount_reducer.py \
-numReduceTasks 2

Now the results will be stored in two files: `/user/cloudera/output_new_2/part-00000` and `/user/cloudera/output_new_2/part-00001`.